In [ ]:
# This code is heavily inspired by this tutorial: https://www.kaggle.com/ab971631/beginners-guide-to-text-generation-pytorch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
import unidecode
import random
import torch
import torch.nn as nn

#from architecture import recursive

from shutil import copyfile
copyfile(src = "../input/recursive/recursive.py", dst = "../working/recursive.py")
import recursive

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cuda':
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(SEED)
print(device)

In [ ]:
#df_train = pd.read_csv("data/lyrics.csv/lyrics.csv") 
df_train = pd.read_csv("../input/lyrics/lyrics.csv") 

In [ ]:
i=0
pop_lyrics = list()
while i < len(df_train.index):
    if df_train['genre'][i] == 'Pop' and type(df_train['lyrics'][i]) == str:
        pop_lyrics.append(df_train['lyrics'][i])
    i += 1

def joinStrings(text):
    return ' '.join(string for string in text)

pop_text = joinStrings(pop_lyrics[:10])
len(pop_text.split())

In [ ]:
stop = set(nltk.corpus.stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = nltk.stem.wordnet.WordNetLemmatizer()
def clean(doc):
        stop_free = " ".join([i for i in doc.split() if i not in stop])
        punc_free = "".join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return normalized
test_sentence = clean(pop_text).lower().split()

In [ ]:
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
chunk_len=len(trigrams)
print(trigrams[:3])

In [ ]:
vocab = set(test_sentence)
voc_len=len(vocab)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [ ]:
inp=[]
tar=[]
for context, target in trigrams:
    context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
    inp.append(context_idxs)
    targ = torch.tensor([word_to_ix[target]], dtype=torch.long)
    tar.append(targ)

In [ ]:
def train(inpt, target, deco, deco_optimizer, chunklen):
    if device == 'cuda':
        hidden = deco.init_hidden().cuda()
    else:
        hidden = deco.init_hidden()
    deco.zero_grad()
    loss = 0
    
    for c in range(chunklen):
        if device == 'cuda':
            output, hidden = deco(inpt[c].cuda(), hidden)
            loss += criterion(output, target[c].cuda())
        else:
            output, hidden = deco(inpt[c], hidden)
            loss += criterion(output, target[c])

    loss.backward()
    deco_optimizer.step()

    return loss.data.item() / chunklen

In [ ]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
n_epochs = 300
print_every = 100
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.015

decoder = recursive.RNN(voc_len, hidden_size, voc_len, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0
if device == 'cuda':
    decoder.cuda()
for epoch in range(1, n_epochs + 1):
    loss = train(inp, tar, decoder, decoder_optimizer, chunk_len)       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 50, loss))
#         print(evaluate('ge', 200), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

plt.figure()
plt.plot(all_losses)

In [ ]:
#torch.save(decoder.state_dict(), "model/trigram_pop_10.pth")
torch.save(decoder.state_dict(), "trigram_pop_10.pth")

## All pop songs

Let's now do the same for all pop songs! I'll write a function so we can use this for other parts of the data set as well.

In [ ]:
def prepare(data):
    text = joinStrings(data)
    test_sentence = clean(text).lower().split()
    trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
    chunk_len =len(trigrams)
    vocab = set(test_sentence)
    voc_len =len(vocab)
    word_to_ix = {word: i for i, word in enumerate(vocab)}
    inp=[]
    tar=[]
    for context, target in trigrams:
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        inp.append(context_idxs)
        targ = torch.tensor([word_to_ix[target]], dtype=torch.long)
        tar.append(targ)
    return(chunk_len, voc_len, inp, tar)

In [ ]:
chunk_len_allpop, voc_len_allpop, inp_allpop, tar_allpop = prepare(pop_lyrics)

In [ ]:
n_epochs_allpop = 10
print_every_allpop = 100
plot_every_allpop = 10
hidden_size_allpop = 100
n_layers_allpop = 1
lr_allpop = 0.015

decoder_allpop = recursive.RNN(voc_len_allpop, hidden_size, voc_len_allpop, n_layers_allpop)
decoder_optimizer_allpop = torch.optim.Adam(decoder_allpop.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses_allpop = []
loss_avg_allpop = 0
if device == 'cuda':
    decoder_allpop.cuda()
for epoch in range(1, n_epochs_allpop + 1):
    loss = train(inp_allpop, tar_allpop, decoder_allpop, decoder_optimizer_allpop, chunk_len_allpop)       
    loss_avg_allpop += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs_allpop * 50, loss))

    if epoch % plot_every == 0:
        all_losses_allpop.append(loss_avg_allpop / plot_every)
        loss_avg_allpop = 0

In [ ]:
%matplotlib inline

plt.figure()
plt.plot(all_losses_allpop)

In [ ]:
#torch.save(decoder_allpop.state_dict(), "model/trigram_allpop.pth")
torch.save(decoder_allpop.state_dict(), "trigram_allpop.pth")